In [4]:
import sqlite3
import time
from bs4 import BeautifulSoup
from requests import get
from abc import ABC
import Levenshtein as lev
import string

In [94]:
conn = sqlite3.connect("bets_database.db")
cursor = conn.cursor()

#If the table does not exists, we create it
if cursor.execute("PRAGMA table_info(TENNIS_PLAYERS)").fetchall() == []:
    tennis_players_table_query = """CREATE TABLE TENNIS_PLAYERS (
        name TEXT     UNIQUE,
        sex  TEXT (1) 
    );"""
    cursor.execute(tennis_players_table_query)
    conn.commit()

In [98]:
def get_ranking(page, sex):
    response = get(page)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find(attrs={'id':'u868'})
    table = table.findAll("tbody")[10]
    for elem, i in zip(table, range(len(table))):
        if i % 2 == 0:
            continue
        player = elem.find("td", attrs={'class':'pn'})
        ranking = elem.find("td", attrs={'class':'rk'})
        if player == None or ranking == None:
            continue
        query = f"INSERT INTO TENNIS_PLAYERS VALUES (\"{player.text}\", \"{sex}\")"
        try:
            cursor.execute(query)
        except:
            continue

In [100]:
%%time
get_ranking("https://live-tennis.eu/en/atp-live-ranking", "H")
get_ranking("https://live-tennis.eu/en/wta-live-ranking", "W")
conn.commit()

Wall time: 4.62 s
